In [2]:
ENV["PYTHONPATH"]="/home/gridsan/aligho/.local/lib/python3.8/site-packages/PyNormaliz-2.15-py3.8-linux-x86_64.egg";

In [3]:
using Crystalline, MPBUtils, SymmetryBases, JLD2, PyPlot, HDF5

In [33]:
# Check permittivity and permeability 
sgs = [2, 6, 9:17...]
for sg in sgs
    println("Analyzing plane group: $sg"); flush(stdout)
    for mode in ["te", "tm"]
        for idx in 1:10
            dir = "./"
            original_dir = "../symeigs/output/sg$sg/eps3/$mode/"
            epsilon = h5open(dir*"dim2-sg$sg-$(idx)-res64-$mode-epsilon.h5", "r") do file
                read(file, "data")
            end
            mu = h5open(dir*"dim2-sg$sg-$idx-res64-$mode-mu.h5", "r") do file
                read(file, "data")
            end;
            epsilon_original = h5open(original_dir*"dim2-sg$sg-$(idx+20000)-res64-$mode-epsilon.h5", "r") do file
                read(file, "data")
            end
            @assert isapprox(epsilon, ones(64, 64))
            @assert isapprox(mu, epsilon_original)

        end
    end
end

Analyzing plane group: 2
Analyzing plane group: 6
Analyzing plane group: 9
Analyzing plane group: 10
Analyzing plane group: 11
Analyzing plane group: 12
Analyzing plane group: 13
Analyzing plane group: 14
Analyzing plane group: 15
Analyzing plane group: 16
Analyzing plane group: 17


In [29]:
sg = 10
idx = 10
mode = "te"
dir = "./"
original_dir = "../symeigs/output/sg$sg/eps3/$mode/"

mu = h5open(dir*"dim2-sg$sg-$idx-res64-$mode-mu.h5", "r") do file
                read(file, "data")
            end
epsilon_original = h5open(original_dir*"dim2-sg$sg-$(idx+20000)-res64-$mode-epsilon.h5", "r") do file
                read(file, "data")
end;

In [124]:
function em_duality_transform(n::Vector{<:Integer}, sg::Integer)
    n_prime = if sg in [2, 10, 13, 16]
        n
    elseif sg == 6
        [n[2], n[1], n[4], n[3], n[6], n[5], n[8], n[7], n[10], n[9], n[12], n[11], n[14], n[13], n[16], n[15], n[17]]
    elseif sg == 9
        [n[2], n[1], n[4], n[3], n[5], n[6], n[8], n[7], n[10], n[9], n[11]]
    elseif sg == 11
        [n[2], n[1], n[4], n[3], n[8], n[7], n[6], n[5], n[9], n[13], n[12], n[11], n[10], n[14], n[15]]
    elseif sg == 12
        [n[2], n[1], n[3], n[4], n[8], n[7], n[6], n[5], n[9], n[10]]
    elseif sg == 14
        [n[2], n[1], n[4], n[3], n[5], n[6], n[7], n[8], n[9]]
    elseif sg == 15
        [n[2], n[1], n[3], n[5], n[4], n[7], n[6], n[8], n[10], n[9], n[11], n[12]]
    elseif sg == 17
        [n[2], n[1], n[4], n[3], n[6], n[5], n[8], n[7], n[9], n[10] , n[12], n[11], n[13], n[14]]
    end
    return n_prime
end;

In [127]:
# Below we check that we should get the exact same topologies upon the em_duality transformation


In [126]:
for sg in [6, 9, 11, 12, 14, 15, 17]
    ns, brs = compatibility_basis(sg, 2)
    for n in ns
        n_prime = em_duality_transform(n, sg)
        @assert n_prime in ns
        original_topo = calc_detailed_topology(n, brs)
        transformed_topo = calc_detailed_topology(n, brs)
        @assert original_topo == transformed_topo
    end
end